In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [2]:
def makeLabelData(column):
    first_cut = column.quantile(0.25)
    second_cut = column.quantile(0.5)
    third_cut = column.quantile(0.75)
    for i in range(0, len(column)):
        if (column[i] < first_cut):
            column[i] = "vl"
        elif (column[i] < second_cut):
            column[i] = "l"
        elif (column[i] < third_cut):
            column[i] = "h"
        else:
            column[i] = "vh"


In [3]:
def fit(X, Y):
    counts = {}
    class_values = set(Y)
    for current_class in class_values:
        counts[current_class] = {}
        rows_with_current_class = (Y == current_class)
        X_current_class = X[rows_with_current_class]
        Y_current_class = Y[rows_with_current_class]
        counts[current_class]["total_count"] = len(Y_current_class)
        for i in range(X_current_class.shape[1]):
            counts[current_class][i] = {}
            feature_vals = set(X[:,i])
            for val in feature_vals:
                counts[current_class][i][val] = (X_current_class[:,i] == val).sum()
    return counts

In [4]:
def findSingleClassProb(x, counts, c):
    ans = 1
    pr_class = counts[c]["total_count"]
    ans = ans * pr_class
    for i in range(len(x)):
        #pr_this_feature = counts[c][i][x[i]]/counts[c]["total_count"]
        a = len(counts[c][i].keys())
        pr_this_feature = (counts[c][i][x[i]] + 1)/(a + counts[c]["total_count"])
        ans = ans * pr_this_feature
    return ans

In [5]:
def predictSingle(x, counts):
    max_pr = 0
    max_class = None
    classes = counts.keys()
    for c in classes:
        p = findSingleClassProb(x, counts, c)
        if p > max_pr:
            max_pr = p
            max_class = c
    return max_class

In [6]:
def predict(x_test, counts):
    output = []
    for x in x_test:
        output.append(predictSingle(x, counts))
    return output

In [7]:
import pandas as pd
df = pd.DataFrame(X)

In [8]:
for i in range(X.shape[1]):
    makeLabelData(df[i])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: 

In [9]:
counts = fit(df.values, Y)
y_pred = predict(df.values, counts)

In [10]:
counts

{0: {'total_count': 50,
  0: {'vl': 28, 'vh': 0, 'l': 21, 'h': 1},
  1: {'vl': 1, 'vh': 33, 'l': 1, 'h': 15},
  2: {'vl': 37, 'vh': 0, 'l': 13, 'h': 0},
  3: {'vl': 34, 'vh': 0, 'l': 16, 'h': 0}},
 1: {'total_count': 50,
  0: {'vl': 3, 'vh': 11, 'l': 18, 'h': 18},
  1: {'vl': 21, 'vh': 2, 'l': 13, 'h': 14},
  2: {'vl': 0, 'vh': 1, 'l': 25, 'h': 24},
  3: {'vl': 0, 'vh': 1, 'l': 15, 'h': 34}},
 2: {'total_count': 50,
  0: {'vl': 1, 'vh': 31, 'l': 2, 'h': 16},
  1: {'vl': 11, 'vh': 8, 'l': 10, 'h': 21},
  2: {'vl': 0, 'vh': 41, 'l': 0, 'h': 9},
  3: {'vl': 0, 'vh': 45, 'l': 0, 'h': 5}}}

In [11]:
from sklearn.metrics import confusion_matrix

In [14]:
confusion_matrix(y_pred=y_pred, y_true=Y)

array([[50,  0,  0],
       [ 0, 48,  2],
       [ 0,  3, 47]])

In [15]:
from sklearn.metrics import classification_report

In [16]:
print(classification_report(y_pred=y_pred, y_true=Y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.94      0.96      0.95        50
           2       0.96      0.94      0.95        50

   micro avg       0.97      0.97      0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150

